In [ ]:
import os
import json
import altair as alt

In [ ]:
runtime_files = []
for f in os.listdir('tracking_results_otif'):
    if f.startswith('runtime_'):
        runtime_files.append(f)

runtime_files.sort()
for f in runtime_files:
    print(f)

In [ ]:
acc_files = []
for f in os.listdir('track-accuracy-eval-pack-otif/xsort'):
    acc_files.append(f)
acc_files.sort()
for f in acc_files:
    print(f)

In [ ]:
benchmark_data = []
for r, a in zip(runtime_files, acc_files):
    print(r, a)
    assert r[len('runtime_'):-len('.json')] == a[len('tracked_'):]
    with open('tracking_results_otif/' + r) as rf:
        with open(os.path.join('track-accuracy-eval-pack-otif/xsort', a, 'car_summary.txt')) as af:
            runtime = json.load(rf)
            acc_k = af.readline().strip().split(' ')
            acc_v = af.readline().strip().split(' ')
            acc = dict(zip(acc_k, acc_v))
            opt_type = r[len('runtime_'):-len('.json')].split('_')[0]
            if '_isr_' in  r[len('runtime_'):-len('.json')]:
                opt_type = 'op + frame skip'
            benchmark_data.append({
                **runtime,
                **acc,
                "expr": r[len('runtime_'):-len('.json')],
                "opt_type": opt_type,
            })
benchmark_data

In [ ]:
alt.Chart(alt.Data(values=benchmark_data)).mark_point().encode(
    x=alt.X('fps:Q', title='fps'),
    y=alt.Y('AssA:Q', title='HOTA'),
    color=alt.Color('opt_type:N', title='Optimization Type'),
    tooltip=[
        alt.Tooltip('expr:N', title='Expression'),
        alt.Tooltip('time:Q', title='Runtime (ms)'),
        alt.Tooltip('fps:Q', title='FPS'),
        alt.Tooltip('AssA:Q', title='AssA'),
        alt.Tooltip('HOTA:Q', title='HOTA'),
    ],
).properties(
    width=500,
    height=500,
)